<a href="https://colab.research.google.com/github/RealGIS-creator/scripts_realgis_banistmo_2025/blob/main/preparacion_datossucursales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

def remove_special_chars_but_keep_accents(text):

    text = re.sub(r'[^\w\sáéíóúÁÉÍÓÚñÑ]', '', text, flags=re.UNICODE)
    return text.upper()

try:
    df = pd.read_csv('/content/ACTUALIZAR_SEGUNDA_ENTREGA.csv', encoding='utf-8', on_bad_lines='warn')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('ACTUALIZAR_SEGUNDA_ENTREGA.csv', encoding='latin-1', on_bad_lines='warn')
    except UnicodeDecodeError:
        df = pd.read_csv('ACTUALIZAR_SEGUNDA_ENTREGA.csv', encoding='ISO-8859-1', on_bad_lines='warn')


df.columns = [''.join(filter(str.isalnum, col)) for col in df.columns]

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).apply(remove_special_chars_but_keep_accents)
        df[col] = df[col].replace('NAN', '', regex=False)

if 'RUC' in df.columns:
    split_ruc = df['RUC'].str.split(' ', expand=True, n=3)
    num_cols = split_ruc.shape[1]
    ruc_cols = [f'RUC_Part{i+1}' for i in range(num_cols)]
    df[ruc_cols] = split_ruc


df.to_csv('/content/DatosSucursalesLimpios.csv', index=False)



In [ ]:
import pandas as pd
import re

# Cargar archivos
acreditados_df = pd.read_csv('ACREDITADOS.csv')
sucursales_df = pd.read_csv('/content/DatosSucursalesLimpios.csv')

# Normalizar identificadores
acreditados_df['acreditadoidenti'] = acreditados_df['acreditadoidenti'].astype(str).str.replace('-', '').str.strip()

# Merge de datos
merged_df = pd.merge(acreditados_df, sucursales_df, left_on='acreditadoidenti', right_on='RUC_Part2', how='inner')

# Limpiar columnas de dirección
for field in ['Ubicacion', 'Urbanizacion', 'Calle', 'Casa', 'Edificio', 'Apartamento']:
    merged_df[field] = merged_df[field].fillna('').str.replace('NAN', '', regex=True)

# Función para eliminar palabras consecutivas duplicadas dentro del texto
def remove_duplicate_words(text):
    words = text.split()
    result = [words[0]] if words else []
    for word in words[1:]:
        if word != result[-1]:
            result.append(word)
    return ' '.join(result)

# Función para generar dirección completa sin duplicados consecutivos de palabras
def get_address(row):
    parts = []
    for field in ['Ubicacion', 'Urbanizacion', 'Calle', 'Casa', 'Edificio', 'Apartamento']:
        value = row[field]
        if pd.notna(value):
            value_clean = value.strip()
            if value_clean:
                parts.append(value_clean)
    address = ', '.join(parts)
    return remove_duplicate_words(address)

# Aplicar dirección completa
merged_df['DireccionCompleta'] = merged_df.apply(get_address, axis=1)

# Eliminar duplicados de RUC
merged_df = merged_df[~merged_df.duplicated(subset=['RUC'], keep=False)]

# Guardar resultado
merged_df.to_csv('/content/merge_panamaemprende_2010_2011.csv', index=False)



KeyError: 'RUC_Part2'

In [ ]:
# PARA SACAR LOS NO COINCIDENTES

acreditados_df = pd.read_csv('ACREDITADOS.csv')
sucursales_df = pd.read_csv('DatosSucursalesLimpios.csv')

acreditados_df['acreditadoidenti'] = acreditados_df['acreditadoidenti'].astype(str).str.replace('-', '').str.strip()

merged_df = pd.merge(acreditados_df, sucursales_df, left_on='acreditadoidenti', right_on='RUC_Part2', how='left')

merged_df['Ubicacion'] = merged_df['Ubicacion'].fillna('').str.replace('NAN','',regex=True)
merged_df['Urbanizacion'] = merged_df['Urbanizacion'].fillna('').str.replace('NAN','',regex=True)
merged_df['Calle'] = merged_df['Calle'].fillna('').str.replace('NAN','',regex=True)
merged_df['Casa'] = merged_df['Casa'].fillna('').str.replace('NAN','',regex=True)
merged_df['Edificio'] = merged_df['Edificio'].fillna('').str.replace('NAN','',regex=True)
merged_df['Apartamento'] = merged_df['Apartamento'].fillna('').str.replace('NAN','',regex=True)

def get_address(row):
  address = ""
  if row['Ubicacion'] and pd.notna(row['Ubicacion']) and row['Ubicacion'] != '':
    address += f"{row['Ubicacion']}, "
  if row['Urbanizacion'] and pd.notna(row['Urbanizacion']) and row['Urbanizacion'] != '':
    address += f"{row['Urbanizacion']}, "
  if row['Calle'] and pd.notna(row['Calle']) and row['Calle'] !='':
      address += f"{row['Calle']}, "
  if row['Casa'] and pd.notna(row['Casa']) and row['Casa'] != '':
      address += f"{row['Casa']}, "
  if row['Edificio'] and pd.notna(row['Edificio']) and row['Edificio'] != '':
    address += f"{row['Edificio']}, "
  if row['Apartamento'] and pd.notna(row['Apartamento']) and row['Apartamento'] != '':
      address += f"{row['Apartamento']}, "
  return address.strip(', ')

merged_df['DireccionCompleta'] = merged_df.apply(get_address, axis=1)
merged_df = merged_df[~merged_df.duplicated(subset=['acreditadoidenti'], keep=False)]

print(merged_df.head())

merged_df.to_csv('/content/datosnocruzados_panamaemprende_2024_2020.csv', index= False)

<ipython-input-3-8dd89faed668>:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  sucursales_df = pd.read_csv('DatosSucursalesLimpios.csv')


   acreditado_id  acreditadonum acreditadoidenti  \
0          42762         6397.0          3732532   
1          25007     98534398.0          4714983   
2          29807        26719.0          4139181   
3          29795        80986.0         82232117   
4          46925       102592.0          N201953   

                            acreditadonom  tipopersona_id  acreditadonumcuen  \
0                      ADRIANO CHIN CANTO               3   5401720000118230   
1            GUSTAVO ARIEL  ROJAS MIRANDA               1             294850   
2                 OVIDIO  REYES RODRIGUEZ               3             425534   
3             GUSTAVO LUIS  ARROYO DUARTE               3             287293   
4  JACQUELINE DEL SOCORRO  ESTRADA ARANGO               3             441661   

                     acrefechcre acreusucre acreipcre  \
0  2025-04-03 17:17:41.760 -0500      admin   0.0.0.0   
1  2025-04-03 17:17:41.760 -0500      admin   0.0.0.0   
2  2025-04-03 17:17:41.760 -0500   